# sumaryzacja tekstu

In [ ]:
from google.colab import drive


drive.mount("/content/drive", force_remount=True)

In [ ]:
!nvidia-smi

In [ ]:
!pip install "datasets<3.0.0"


In [ ]:
!pip install bert-extractive-summarizer

In [ ]:
!pip install pytorch-ignite
!pip install torchmetrics

## importowanie datesetu

In [ ]:
from datasets import load_dataset


ds = load_dataset("EdinburghNLP/xsum", trust_remote_code=True)
ds= ds.filter(lambda example: len(example["document"]) < 1500)
split_result = ds["test"].train_test_split(test_size=2, seed=42)
shots = split_result["test"]
ds["test"] = split_result["train"]

In [ ]:
print(ds)
print(shots)

In [ ]:
from summarizer import Summarizer
model_bertsum = Summarizer()

In [ ]:
from ignite.metrics import Rouge

m = Rouge(variants=["L", 1], multiref="best")

candidate = "the cat is not there".split()
references = [
    "the cat is on the mat".split(),
    "there is a cat on the mat".split()
]

m.update(([candidate], [references]))

print(m.compute())

In [ ]:
num_workers = 1
batch_size = 1

In [ ]:
from torch.utils.data import DataLoader
import torch
test_dataset = ds["test"].select(range(50))
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
print(test_dataset)


In [ ]:
import string


def clean(text):
    if not text:
        return ["."]
    text = text.lower().translate(
        str.maketrans(string.punctuation, " " * len(string.punctuation))
    )
    return text.split()


def rouge_score_loss(
    batch,
    model,
    bert: bool,
    tokenizer: any,
    prompt=None,
    promptsuff=None,
    m=Rouge(variants=["L", 2], multiref="best"),
    prints = False,
    chat = False
):
    if bert:
        predictions = [
            model(doc, ratio=0.1) for doc in batch["document"]
        ]
    else:
        docs = [doc for doc in batch["document"]]

        prompted = [
            f"{prompt}{doc}{promptsuff}" for doc in batch["document"]
        ]
        if chat:
          prompted = [
            {"role": "user", "content": prompt}
          ]

          text = tokenizer.apply_chat_template(
              prompted,
              tokenize=False,
              add_generation_prompt=True
          )
          inputs = tokenizer([text], return_tensors="pt").to(model.device)
        else:
          inputs = tokenizer(
              prompted,
              return_tensors="pt",
              padding=True,
              truncation=True,
              max_length=4096,
          )

        device = next(model.parameters()).device
        inputs = {k: v.to(device) for k, v in inputs.items()}

        summary_ids = model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=50,
            do_sample=True,
            top_k=0,
            temperature=0.5,
        )

        input_length = inputs["input_ids"].shape[1]
        new_tokens = summary_ids[:, input_length:]

        predictions = tokenizer.batch_decode(
            new_tokens, skip_special_tokens=True
        )

    if prints:
      # print(f"Doc: {batch['document'][0]}")
      print(f"Prediction: {predictions[0]}")
      # print(f"Target: {batch['summary'][0]}")

    predictions = [
        clean(pred) if pred.strip() else "." for pred in predictions
    ]

    targets = [[clean(ref)] for ref in batch["summary"]]
    m.update((predictions, targets))
    # print(m.compute())
    return m.compute()

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

def plot_data(data, labels=['Rouge-1-P', 'Rouge-1-R', 'Rouge-1-F','Rouge-2-P', 'Rouge-2-R', 'Rouge-2-F',  'Rouge-L-P', 'Rouge-L-R', 'Rouge-L-F', ], models = ['Qwen 1.5 0.5B - LoRa', 'Qwen 1.5 0.5B - few shot 1', 'Qwen 1.5 0.5B - few shot 3', 'Qwen 1.5 0.5B - zero shot', 'Bertsum - bert-base-uncased', 'Bertsum - Sbertmodel', 'Bertsum - roberta-base','Qwen 2.5 3B - zero shot', 'Qwen 2.5 3B - few shot 1',  'Qwen 2.5 3B - few shot 3']):
    x = np.arange(len(labels))
    width = 0.075

    plt.figure(figsize=(40, 12))
    for i, row in enumerate(data):
        offset = x + (i * width)
        plt.bar(offset, row, width=width, label=models[i])
    center_offset = ((len(data) - 1) * width) / 2
    plt.xticks(x + center_offset, labels)

    plt.ylim(0, 1)
    plt.ylabel('Score')
    plt.title('ROUGE Score Comparison')
    plt.legend()
    plt.show()

In [ ]:
def batch_to_device(batch: dict, device) -> dict:
    return {k: v.to(device) for k, v in batch.items()}

In [ ]:
import torch
import torch.nn as nn
from torch import Tensor
def calc_loss_batch(batch: dict, model: nn.Module) -> Tensor:
    logits = model(batch['input_ids'])
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), batch['labels'].flatten())
    return loss

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
def calc_loss_loader(
    data_loader,
    model,
    device,
    bert: bool,
    tokenizer: any,
    prompt=None,
    promptsuff=None,
    prints = False,
    chat=False
) -> float:
    results = []
    length = len(data_loader)
    for i, batch in enumerate(data_loader):
        m = Rouge(variants=[1,2,"L"], multiref="best")
        loader_bar = tqdm(data_loader, desc="Calculating ROUGE average score", leave=False)
        ret = rouge_score_loss(
            batch,
            model,
            bert,
            tokenizer,
            prompt,
            promptsuff,
            m,
            prints,
            chat,
        )
        results.append(list(ret.values()))

    results = np.array(results)
    return np.mean(results, axis=0)


In [ ]:
def evaluate_model(
    model: nn.Module,
    loader: DataLoader,
    device,
    bert: bool,
    tokenizer: any,
    prompt: str = "",
    promptsuff: str = "",
    prints = False,
    chat = False
) -> tuple[float, float]:
    if not bert:
        model.eval()

    with torch.no_grad():
        loss = calc_loss_loader(
            loader,
            model,
            device,
            bert=bert,
            tokenizer=tokenizer,
            prompt=prompt,
            promptsuff=promptsuff,
            prints=prints,
            chat=chat
        )

    return loss




In [ ]:
from torch import nn
data_bertsum = evaluate_model(model_bertsum, test_loader, device='cuda', bert=True, tokenizer=None)



In [ ]:
from summarizer.sbert import SBertSummarizer
model_sbert = SBertSummarizer('paraphrase-MiniLM-L6-v2')
data_bertsum_sbert = evaluate_model(model_sbert, test_loader, device='cuda', bert=True, tokenizer=None)

In [ ]:
from summarizer import TransformerSummarizer
model_roberta = TransformerSummarizer(transformer_type="Roberta", transformer_model_key="roberta-base")
data_bertsum_roberta = evaluate_model(model_roberta, test_loader, device='cuda', bert=True, tokenizer=None)

In [ ]:
def generate_prompt(tokenizer, size):
    prompt = ""
    shots = ds["train"].train_test_split(train_size=size, seed=35)["train"]
    shots_loader = DataLoader(shots, batch_size=100000, shuffle=True, num_workers=num_workers)
    for batch in shots_loader:
        for i in range(len(batch['document'])):
            prompt += "[USER] Summarize this text. Don't hallucinate. Be thorough and explicit. Always provide an anwer."
            prompt += f'[TEXT]{batch['document'][i]}[AGENT]{batch['summary'][i]}'
            prompt += tokenizer.eos_token + "\n"
    prompt += "[USER] Summarize this text. Don't hallucinate. Be thorough and explicit. Always provide an anwer.[TEXT]"
    return prompt


In [ ]:
!pip uninstall -y unsloth unsloth_zoo

# 2. Install Unsloth Zoo DIRECTLY from Git (to get 'tiled_mlp')
!pip install --upgrade --no-cache-dir "unsloth_zoo @ git+https://github.com/unslothai/unsloth-zoo.git"

# 3. Install Unsloth main library from Git
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from unsloth import FastLanguageModel



In [ ]:
import gc
try:
  del model
  del tokenizer3
  del model
except NameError:
  pass

gc.collect()

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen1.5-0.5B", max_seq_length=32000, load_in_4bit=False
)
prompt1 = generate_prompt(tokenizer, 1)
print(prompt1)
torch.cuda.empty_cache()
data_1 = evaluate_model(model, test_loader, device='cuda', bert=False, tokenizer=tokenizer, prompt=prompt1, promptsuff = "[AGENT]")

In [ ]:

prompt3 = generate_prompt(tokenizer, 3)
torch.cuda.empty_cache()
data_3 = evaluate_model(model, test_loader, device='cuda', bert=False, tokenizer=tokenizer, prompt=prompt3, promptsuff = "[AGENT]")

In [ ]:
prompt0 = "[USER] Summarize this text. Don't hallucinate. Be thorough and explicit. Always provide an anwer.[TEXT]"
data_zero_shot = evaluate_model(
        model,
        test_loader,
        device="cuda",
        bert=False,
        tokenizer=tokenizer,
        prompt=prompt0,
        promptsuff = "[AGENT]"
    )


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from peft import PeftModel
model_path = './drive/MyDrive/models/ft'
ft_model = PeftModel.from_pretrained(model, model_path, max_sequence_length=32000, load_in_4bit=False)


In [ ]:
data_ft = evaluate_model(ft_model, test_loader, 'cuda', bert=False, tokenizer = tokenizer, prompt = "[USER] Summarize this text. Don't hallucinate. Be thorough and explicit. Always provide an anwer.[TEXT]", promptsuff = "[AGENT]")

In [ ]:
prompt_instruct = "Summarize this text. Don't hallucinate. Be thorough and explicit. Always provide an anwer."
model15_instruct, tokenizer_instruct_15 = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen1.5-0.5B-chat", max_seq_length=32000, load_in_4bit=False
)
data_instruct_15 = evaluate_model(model15_instruct, test_loader, device='cuda', bert=False, tokenizer=tokenizer_instruct_15, prompt=prompt_instruct, promptsuff = "[AGENT]", chat=True)

In [ ]:
model25_instruct, tokenizer_instruct_25 = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen2.5-0.5B-instruct", max_seq_length=32000, load_in_4bit=False
)
data_instruct_25 = evaluate_model(model15_instruct, test_loader, device='cuda', bert=False, tokenizer=tokenizer_instruct_25, prompt=prompt_instruct, promptsuff = "[AGENT]", chat=True)

In [ ]:
model_better, tokenizer_better = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen2.5-3B", max_seq_length=32000, load_in_4bit=False
)
torch.cuda.empty_cache()
data_better = evaluate_model(model_better, test_loader, device='cuda', bert=False, tokenizer=tokenizer_better, prompt=prompt0, promptsuff = "[AGENT]")

In [ ]:
prompt1_better = generate_prompt(tokenizer_better, 1)
data_better_1 = evaluate_model(model_better, test_loader, device='cuda', bert=False, tokenizer=tokenizer_better, prompt=prompt1_better, promptsuff = "[AGENT]")

In [ ]:
prompt3_better = generate_prompt(tokenizer_better, 3)
data_better_3 = evaluate_model(model_better, test_loader, device='cuda', bert=False, tokenizer=tokenizer_better, prompt=prompt3_better, promptsuff = "[AGENT]")

In [ ]:
plot_data([data_ft, data_1,data_3, data_zero_shot,data_instruct_15, data_bertsum, data_bertsum_sbert,data_bertsum_roberta, data_better, data_better_1, data_better_3, data_instruct_25])


In [ ]:
import random
random_index = random.randint(0, len(ds["test"]) - 1)
showcase_dataset = ds["test"].select([random_index])
showcase_loader = DataLoader(showcase_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
batch = next(iter(showcase_loader))
print(f"Document: {batch['document'][0]}")
print(f"Target: {batch['summary'][0]}")
print("Qwuen 1.5 0.5B zero shot:")
evaluate_model(model, showcase_loader , device='cuda', bert=False, tokenizer=tokenizer, prompt=prompt0, promptsuff = "[AGENT]", prints = True)
print("Qwuen 1.5 0.5B one shot:")
evaluate_model(model, showcase_loader , device='cuda', bert=False, tokenizer=tokenizer, prompt=prompt1, promptsuff = "[AGENT]", prints = True)
print("Qwuen 1.5 0.5B three shot:")
evaluate_model(model, showcase_loader , device='cuda', bert=False, tokenizer=tokenizer, prompt=prompt3, promptsuff = "[AGENT]", prints = True)
print("Qwuen 1.5 0.5B fine tuned:")
evaluate_model(ft_model, showcase_loader , device='cuda', bert=False, tokenizer=tokenizer, prompt=prompt0, promptsuff = "[AGENT]", prints = True)
print("Qwuen 1.5 0.5B chat:")
evaluate_model(model15_instruct, test_loader, device='cuda', bert=False, tokenizer=tokenizer_instruct_15, prompt=prompt_instruct, chat=True)
print("Qwuen 2.5 0.5B instruct:")
evaluate_model(model25_instruct, test_loader, device='cuda', bert=False, tokenizer=tokenizer_instruct_25, prompt=prompt_instruct, chat=True)
print("Qwuen 2.5 3B zero shot:")
evaluate_model(model_better, showcase_loader , device='cuda', bert=False, tokenizer=tokenizer_better, prompt=prompt0, promptsuff = "[AGENT]", prints = True)
print("Qwuen 2.5 3B one shot:")
evaluate_model(model_better, showcase_loader , device='cuda', bert=False, tokenizer=tokenizer_better, prompt=prompt1_better, promptsuff = "[AGENT]", prints = True)
print("Qwuen 2.5 3B three shot:")
evaluate_model(model_better, showcase_loader , device='cuda', bert=False, tokenizer=tokenizer_better, prompt=prompt3_better, promptsuff = "[AGENT]", prints = True)
print('Bertsum - bert-base-uncased')
evaluate_model(model_bertsum, showcase_loader, device='cuda', bert=True, tokenizer=None, prints = True)
print('Bertsum - Sbertmodel')
evaluate_model(model_sbert, showcase_loader, device='cuda', bert=True, tokenizer=None, prints = True)
print('Bertsum - roberta-base')
evaluate_model(model_roberta, showcase_loader, device='cuda', bert=True, tokenizer=None, prints = True)

In [ ]:
data_qwen15 = np.array([data_zero_shot, data_1, data_3, data_ft])
data_bertsums = np.array([data_bertsum, data_bertsum_sbert, data_bertsum_roberta])
data_qwen25 = np.array([data_better, data_better_1, data_better_3])

print(data_qwen15)

In [ ]:
print(data_bertsums)

In [ ]:
print(data_qwen25)